### Import Required Libraries and Set Up Environment Variables

In [2]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [28]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")
print(nyt_api_key, tmdb_api_key)

F4CZwGlh6Yt5mQAG7IujCt0D3v6oljCd dceb72b869c450fd1494605a36def656


### Access the New York Times API

In [4]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
article_url = f"{url}api-key={nyt_api_key}&fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}"
article_url

'https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=F4CZwGlh6Yt5mQAG7IujCt0D3v6oljCd&fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&begin_date=20130101&end_date=20230531'

In [5]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    request_url = f"{article_url}&page={page + 1}"
    
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(request_url).json()
    
    # Try and save the reviews to the reviews_list
    try:
        # I added this check because even though there is no results the loop continues to be executed
        if len(reviews["response"]["docs"]) == 0:
            raise OverflowError()
        
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
        # Print the page that was just retrieved
        print(f"Checked page {page}")

    except:
        # Print the page number that had no results then break from the loop
        print("Out of bounds")
        print(f"Checked page without results: {page}")
        break

    # NOTE: I moved this line here on purpose. It will not make unnecesary stops before checking if retrieved results.
    # In that case, instead of stopping, it will go to the Out of bounds exception and stop execution.
    # At this point, we are sure is only executed when we need to make another request

    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)

Checked page 0
{'status': 'OK', 'copyright': 'Copyright (c) 2024 The New York Times Company. All Rights Reserved.', 'response': {'docs': [{'web_url': 'https://www.nytimes.com/2023/01/31/movies/pamela-a-love-story-review.html', 'snippet': 'This documentary from Ryan White rewinds, to powerful effect, on Pamela Anderson’s life and fame.', 'source': 'The New York Times', 'headline': {'main': '‘Pamela, a Love Story’ Review: A Frank Look Back', 'kicker': None, 'content_kicker': None, 'print_headline': 'Pamela, a Love Story', 'name': None, 'seo': None, 'sub': None}, 'keywords': [{'name': 'subject', 'value': 'Documentary Films and Programs', 'rank': 1, 'major': 'N'}, {'name': 'persons', 'value': 'Anderson, Pamela (1967- )', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'White, Ryan (Filmmaker)', 'rank': 3, 'major': 'N'}, {'name': 'creative_works', 'value': 'Pamela, a Love Story (Movie)', 'rank': 4, 'major': 'N'}], 'pub_date': '2023-01-31T12:00:05+0000', 'byline': {'original': 'By Gle

In [6]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list, indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/01/31/movies/pamela-a-love-story-review.html",
        "snippet": "This documentary from Ryan White rewinds, to powerful effect, on Pamela Anderson\u2019s life and fame.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018Pamela, a Love Story\u2019 Review: A Frank Look Back",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "Pamela, a Love Story",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Documentary Films and Programs",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "persons",
                "value": "Anderson, Pamela (1967- )",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",

In [7]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
articles_df= pd.json_normalize(reviews_list)
articles_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/01/31/movies/pame...,"This documentary from Ryan White rewinds, to p...",The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-01-31T12:00:05+0000,295,"‘Pamela, a Love Story’ Review: A Frank Look Back",None,None,"Pamela, a Love Story",None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
1,https://www.nytimes.com/2023/01/19/movies/in-f...,"In Matt Carter’s gay rugby film, sports and ro...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-01-19T17:50:16+0000,281,‘In From the Side’ Review: Love and Rugby Play...,None,None,In From the Side,None,None,None,By Kyle Turner,"[{'firstname': 'Kyle', 'middlename': None, 'la...",None
2,https://www.nytimes.com/2023/01/19/movies/afte...,In this intelligent melodrama by the director ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-01-19T12:00:06+0000,359,‘After Love’ Review: The Other Woman,Critic’s Pick,None,After Love,None,None,None,By Beatrice Loayza,"[{'firstname': 'Beatrice', 'middlename': None,...",None
3,https://www.nytimes.com/2023/01/05/movies/alca...,"In this naturalistic drama from Spain, a famil...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-01-05T12:00:03+0000,306,‘Alcarràs’ Review: Labor of Love,None,None,Alcarràs,None,None,None,By Devika Girish,"[{'firstname': 'Devika', 'middlename': None, '...",None
4,https://www.nytimes.com/2022/12/15/movies/nell...,A family archive provides intimate records of ...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2022-12-15T12:00:04+0000,308,"‘Nelly & Nadine’ Review: An Unlikely Love, an ...",None,None,Nelly &amp; Nadine,None,None,None,By Teo Bugbee,"[{'firstname': 'Teo', 'middlename': None, 'las...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2016/12/08/movies/fran...,"Frankie and Johnny’s story returns, written an...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2016-12-08T22:05:30+0000,554,"Review: ‘Frank & Lola,’ Retreading Old Ground",None,None,Tainted Love in Familiar Packaging,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
196,https://www.nytimes.com/2016/11/22/movies/alli...,“Allied” is not so much a work of art as a tri...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2016-11-22T21:59:34+0000,901,"Review: ‘Allied,’ With Brad Pitt and Marion Co...",None,None,"For This Couple, All’s Ambiguous in Love and War",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None
197,https://www.nytimes.com/2016/11/18/movies/revi...,Anna Biller’s new film stars Samantha Robinson...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2016-11-17T22:14:13+0000,604,"Review: ‘The Love Witch,’ Hell-Bent on Capturi...",None,None,"A Dark Seductress, Hell-Bent on Capturing Hearts",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None
198,https://www.nytimes.com/2016/11/18/movies/ali-...,"During the era surrounding World War I, in the...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2016-11-17T22:11:30+0000,280,"Review: ‘Ali and Nino,’ a Love Story Set Again...",None,None,None,None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None


In [17]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
def extract_title(s):
    early_review = s.find('Review') == 0
    start_at = '\u2018'
    til_at = '\u2019' if early_review else '\u2019 Review'

    start = s.find(start_at) + len(start_at)
    end = s.find(til_at, start)
    sub = s[start:end]

    # Added this to eliminate the comma before the title
    if early_review:
        sub = sub.replace(',', '')

    return sub

articles_df["title"] = articles_df["headline.main"].apply(extract_title)
articles_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/01/31/movies/pame...,"This documentary from Ryan White rewinds, to p...",The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-01-31T12:00:05+0000,295,"‘Pamela, a Love Story’ Review: A Frank Look Back",None,None,"Pamela, a Love Story",None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,"Pamela, a Love Story"
1,https://www.nytimes.com/2023/01/19/movies/in-f...,"In Matt Carter’s gay rugby film, sports and ro...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-01-19T17:50:16+0000,281,‘In From the Side’ Review: Love and Rugby Play...,None,None,In From the Side,None,None,None,By Kyle Turner,"[{'firstname': 'Kyle', 'middlename': None, 'la...",None,In From the Side
2,https://www.nytimes.com/2023/01/19/movies/afte...,In this intelligent melodrama by the director ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-01-19T12:00:06+0000,359,‘After Love’ Review: The Other Woman,Critic’s Pick,None,After Love,None,None,None,By Beatrice Loayza,"[{'firstname': 'Beatrice', 'middlename': None,...",None,After Love
3,https://www.nytimes.com/2023/01/05/movies/alca...,"In this naturalistic drama from Spain, a famil...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-01-05T12:00:03+0000,306,‘Alcarràs’ Review: Labor of Love,None,None,Alcarràs,None,None,None,By Devika Girish,"[{'firstname': 'Devika', 'middlename': None, '...",None,Alcarràs
4,https://www.nytimes.com/2022/12/15/movies/nell...,A family archive provides intimate records of ...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2022-12-15T12:00:04+0000,308,"‘Nelly & Nadine’ Review: An Unlikely Love, an ...",None,None,Nelly &amp; Nadine,None,None,None,By Teo Bugbee,"[{'firstname': 'Teo', 'middlename': None, 'las...",None,Nelly & Nadine
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2016/12/08/movies/fran...,"Frankie and Johnny’s story returns, written an...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2016-12-08T22:05:30+0000,554,"Review: ‘Frank & Lola,’ Retreading Old Ground",None,None,Tainted Love in Familiar Packaging,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Frank & Lola
196,https://www.nytimes.com/2016/11/22/movies/alli...,“Allied” is not so much a work of art as a tri...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2016-11-22T21:59:34+0000,901,"Review: ‘Allied,’ With Brad Pitt and Marion Co...",None,None,"For This Couple, All’s Ambiguous in Love and War",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None,Allied
197,https://www.nytimes.com/2016/11/18/movies/revi...,Anna Biller’s new film stars Samantha Robinson...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2016-11-17T22:14:13+0000,604,"Review: ‘The Love Witch,’ Hell-Bent on Capturi...",None,None,"A Dark Seductress, Hell-Bent on Capturing Hearts",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None,The Love Witch
198,https://www.nytimes.com/2016/11/18/movies/ali-...,"During the era surrounding World War I, in the...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2016-11-17T22:11:30+0000,280,"Review: ‘Ali and Nino,’ a Love Story Set Again...",None,None,None,None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Ali and Nino


In [19]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
articles_df['keywords'] = articles_df['keywords'].apply(extract_keywords)
articles_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/01/31/movies/pame...,"This documentary from Ryan White rewinds, to p...",The New York Times,subject: Documentary Films and Programs;person...,2023-01-31T12:00:05+0000,295,"‘Pamela, a Love Story’ Review: A Frank Look Back",None,None,"Pamela, a Love Story",None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,"Pamela, a Love Story"
1,https://www.nytimes.com/2023/01/19/movies/in-f...,"In Matt Carter’s gay rugby film, sports and ro...",The New York Times,subject: Movies;creative_works: In From the Si...,2023-01-19T17:50:16+0000,281,‘In From the Side’ Review: Love and Rugby Play...,None,None,In From the Side,None,None,None,By Kyle Turner,"[{'firstname': 'Kyle', 'middlename': None, 'la...",None,In From the Side
2,https://www.nytimes.com/2023/01/19/movies/afte...,In this intelligent melodrama by the director ...,The New York Times,subject: Movies;creative_works: After Love (20...,2023-01-19T12:00:06+0000,359,‘After Love’ Review: The Other Woman,Critic’s Pick,None,After Love,None,None,None,By Beatrice Loayza,"[{'firstname': 'Beatrice', 'middlename': None,...",None,After Love
3,https://www.nytimes.com/2023/01/05/movies/alca...,"In this naturalistic drama from Spain, a famil...",The New York Times,"subject: Movies;persons: Simon, Carla;creative...",2023-01-05T12:00:03+0000,306,‘Alcarràs’ Review: Labor of Love,None,None,Alcarràs,None,None,None,By Devika Girish,"[{'firstname': 'Devika', 'middlename': None, '...",None,Alcarràs
4,https://www.nytimes.com/2022/12/15/movies/nell...,A family archive provides intimate records of ...,The New York Times,subject: Documentary Films and Programs;person...,2022-12-15T12:00:04+0000,308,"‘Nelly & Nadine’ Review: An Unlikely Love, an ...",None,None,Nelly &amp; Nadine,None,None,None,By Teo Bugbee,"[{'firstname': 'Teo', 'middlename': None, 'las...",None,Nelly & Nadine
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2016/12/08/movies/fran...,"Frankie and Johnny’s story returns, written an...",The New York Times,"subject: Movies;persons: Shannon, Michael;crea...",2016-12-08T22:05:30+0000,554,"Review: ‘Frank & Lola,’ Retreading Old Ground",None,None,Tainted Love in Familiar Packaging,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Frank & Lola
196,https://www.nytimes.com/2016/11/22/movies/alli...,“Allied” is not so much a work of art as a tri...,The New York Times,"subject: Movies;persons: Pitt, Brad;persons: C...",2016-11-22T21:59:34+0000,901,"Review: ‘Allied,’ With Brad Pitt and Marion Co...",None,None,"For This Couple, All’s Ambiguous in Love and War",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None,Allied
197,https://www.nytimes.com/2016/11/18/movies/revi...,Anna Biller’s new film stars Samantha Robinson...,The New York Times,subject: Movies;creative_works: The Love Witch...,2016-11-17T22:14:13+0000,604,"Review: ‘The Love Witch,’ Hell-Bent on Capturi...",None,None,"A Dark Seductress, Hell-Bent on Capturing Hearts",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None,The Love Witch
198,https://www.nytimes.com/2016/11/18/movies/ali-...,"During the era surrounding World War I, in the...",The New York Times,"subject: Movies;persons: Hampton, Christopher;...",2016-11-17T22:11:30+0000,280,"Review: ‘Ali and Nino,’ a Love Story Set Again...",None,None,None,None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Ali and Nino


In [20]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = articles_df['title'].drop_duplicates().to_list()
titles

['Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Cloud',
 'A Journal for Jordan',
 'West Side Story',
 'Aulcie',
 'Love Is Love Is Love',
 'Love Hard',
 'Bergman Island',
 'Hard Luck Love Song',
 'South of Heaven',
 'Wife of a Spy',
 'Happier Than Ever',
 'Together',
 'Annette',
 'Resort to Love',
 'Woodstock 99: Peace, Love and Rage',
 'Casan

### Access The Movie Database API

In [38]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie"
tmdb_key_string = "&api_key=" + tmdb_api_key

In [86]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in titles:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        time.sleep(1)
        print("Let me take a break...")

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    good_url = f"{url}?query={title}{tmdb_key_string}"
    movie = requests.get(good_url).json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = movie['results'][0]['id']

        # Make a request for a the full movie details
        movie_detail_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"

        # Execute "GET" request with url
        movie_details = requests.get(movie_detail_url).json()
        # Extract the genre names into a list
        genres = [genre['name'] for genre in movie_details['genres']]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [lang['english_name'] for lang in movie_details['spoken_languages']]

        # Extract the production_countries' name into a list
        production_countries = [country['name'] for country in movie_details['production_countries']]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        data = dict(
            title = movie_details['title'], 
            original_title = movie_details['original_title'], 
            budget = movie_details['budget'], 
            original_language = movie_details['original_language'], 
            homepage = movie_details['homepage'], 
            overview = movie_details['overview'], 
            popularity = movie_details['popularity'], 
            runtime = movie_details['runtime'], 
            revenue = movie_details['revenue'], 
            release_date = movie_details['release_date'], 
            vote_average = movie_details['vote_average'], 
            vote_count = movie_details['vote_count'],
            genres = genres,
            spoken_languages = spoken_languages,
            production_countries = production_countries
        )
        
        tmdb_movies_list.append(data)

        # Print out the title that was found
        print(f"Found {title}")
    except Exception as e:
        print(f"Error procesing {title}: {e}")


Found Pamela, a Love Story
Found In From the Side
Found After Love
Found Alcarràs
Found Nelly & Nadine
Found Lady Chatterley’s Lover
Found The Sound of Christmas
Found The Inspection
Found Bones and All
Found My Policeman
Found About Fate
Found Waiting for Bojangles
Found I Love My Dad
Found A Love Song
Found Alone Together
Found Art of Love
Found The Wheel
Found Thor: Love and Thunder
Found Both Sides of the Blade
Found Fire of Love
Found Love & Gelato
Found Stay Prayed Up
Found Benediction
Found Dinner in America
Found In a New York Minute
Found Anaïs in Love
Found I Love America
Found See You Then
Found La Mami
Found Love After Love
Found Deep Water
Found Lucy and Desi
Found Cyrano
Found The In Between
Found Book of Love
Found Lingui, the Sacred Bonds
Found The Pink Cloud
Found A Journal for Jordan
Found West Side Story
Error procesing Aulcie: list index out of range
Found Love Is Love Is Love
Found Love Hard
Found Bergman Island
Found Hard Luck Love Song
Found South of Heaven
Found

In [76]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list, indent=4))

[
    {
        "title": "Pamela, A Love Story",
        "original_title": "Pamela, A Love Story",
        "budget": 0,
        "original_language": "en",
        "homepage": "https://www.netflix.com/title/81590934",
        "overview": "In her own words, through personal video and diaries, Pamela Anderson shares the story of her rise to fame, rocky romances and infamous sex tape scandal.",
        "popularity": 17.069,
        "runtime": 113,
        "revenue": 0,
        "release_date": "2023-01-30",
        "vote_average": 7.0,
        "vote_count": 209,
        "genres": [
            "Documentary"
        ],
        "spoken_languages": [
            "English"
        ],
        "production_countries": [
            "United States of America"
        ]
    }
]


In [89]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,"Pamela, A Love Story","Pamela, A Love Story",0,en,https://www.netflix.com/title/81590934,"In her own words, through personal video and d...",17.069,113,0,2023-01-30,7.000,209,[Documentary],[English],[United States of America]
1,In from the Side,In from the Side,0,en,http://www.infromthesidemovie.com,"Mark, a new and inexperienced rugby club membe...",16.041,134,52885,2022-09-16,6.714,63,"[Drama, Romance]",[English],[United Kingdom]
2,After Love,After Love,0,en,,Set in the port town of Dover in the South-Eas...,11.242,89,0,2021-06-04,7.164,125,[Drama],"[English, Arabic, French, Urdu]",[United Kingdom]
3,Alcarràs,Alcarràs,3,ca,https://www.alcarras-film.com/,"In a small village in Catalonia, the peach far...",11.197,115,3324521,2022-04-29,6.900,185,[Drama],[Catalan],[Spain]
4,Nelly Knows Mysteries: A Fatal Engagement,Nelly Knows Mysteries: A Fatal Engagement,0,en,https://www.hallmarkmystery.com/nelly-knows-my...,When an unusual handwritten letter arrives at ...,11.516,84,0,2024-08-23,7.667,2,"[Comedy, Mystery, TV Movie]",[English],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,My Best Friend Anne Frank,Mijn beste vriendin Anne Frank,0,nl,https://www.netflix.com/title/81248111,Based on the real-life friendship between Anne...,12.788,103,0,2021-10-07,6.856,240,"[History, War, Drama]","[German, Dutch]",[Netherlands]
192,Allied,Allied,85000000,en,http://www.alliedmovie.com/,"In 1942, an intelligence officer in North Afri...",22.753,124,119520023,2016-02-12,6.792,4468,"[Action, Drama, War, Romance, Thriller]","[Arabic, English, French, German]","[China, United States of America, United Kingdom]"
193,The Love Witch,The Love Witch,0,en,http://www.lifeofastar.com,A modern-day witch uses spells and magic to ge...,12.123,121,228894,2016-11-11,6.419,514,"[Horror, Comedy, Romance]",[English],[United States of America]
194,Ali and Nino,Ali and Nino,0,en,,Muslim prince Ali and Georgian aristocrat Nino...,10.108,100,0,2016-01-27,6.700,72,"[Drama, Romance]","[Russian, English, Azerbaijani]","[Azerbaijan, United Kingdom]"


### Merge and Clean the Data for Export

In [123]:
# Merge the New York Times reviews and TMDB DataFrames on title
df = pd.merge(tmdb_df, articles_df, on='title')
df['spoken_languages']

0        [English, Arabic, French, Urdu]
1        [English, Arabic, French, Urdu]
2                              [Catalan]
3                              [English]
4             [Arabic, Spanish, English]
                     ...                
154                            [English]
155    [Arabic, English, French, German]
156                            [English]
157      [Russian, English, Azerbaijani]
158                            [English]
Name: spoken_languages, Length: 159, dtype: object

In [127]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
characters_to_remove = ['[', ']', "'"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    df[column] = df[column].astype(str)

    # Loop through characters to remove
    for char in characters_to_remove:
        df[column] = df[column].str.replace(char, '', regex=False)

# Display the fixed DataFrame
df

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,After Love,After Love,0,en,,Set in the port town of Dover in the South-Eas...,11.242,89,0,2021-06-04,...,‘After Love’ Review: The Other Woman,Critic’s Pick,None,After Love,None,None,None,By Beatrice Loayza,"[{'firstname': 'Beatrice', 'middlename': None,...",None
1,After Love,After Love,0,en,,Set in the port town of Dover in the South-Eas...,11.242,89,0,2021-06-04,...,Review: Staying Together ‘After Love’ and Regr...,None,None,After Love,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
2,Alcarràs,Alcarràs,3,ca,https://www.alcarras-film.com/,"In a small village in Catalonia, the peach far...",11.197,115,3324521,2022-04-29,...,‘Alcarràs’ Review: Labor of Love,None,None,Alcarràs,None,None,None,By Devika Girish,"[{'firstname': 'Devika', 'middlename': None, '...",None
3,The Sound of Christmas,The Sound of Christmas,0,en,,A woman facing eviction just before Christmas ...,1.854,0,0,2022-11-24,...,‘The Sound of Christmas’ Review: A Gospel Sing...,None,None,The Sound Of Christmas,None,None,None,By Concepción de León,"[{'firstname': 'Concepción', 'middlename': Non...",None
4,The Inspection,The Inspection,0,en,https://a24films.com/films/inspection,"Ellis French is a young, gay Black man, reject...",11.802,95,270613,2022-11-18,...,"‘The Inspection’ Review: Boot Camp, a Love Story",critic’s pick,None,"A Few Good Men, Some With Secrets",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,Passengers,Passengers,110000000,en,http://www.passengersmovie.com/,A spacecraft traveling to a distant colony pla...,47.257,116,303144152,2016-12-21,...,Review: Two ‘Passengers’ Trapped on a Spaceshi...,None,None,Fasten Your Seat Belts for a 120-Year Ride,None,None,None,By Stephen Holden,"[{'firstname': 'Stephen', 'middlename': None, ...",None
155,Allied,Allied,85000000,en,http://www.alliedmovie.com/,"In 1942, an intelligence officer in North Afri...",22.753,124,119520023,2016-02-12,...,"Review: ‘Allied,’ With Brad Pitt and Marion Co...",None,None,"For This Couple, All’s Ambiguous in Love and War",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None
156,The Love Witch,The Love Witch,0,en,http://www.lifeofastar.com,A modern-day witch uses spells and magic to ge...,12.123,121,228894,2016-11-11,...,"Review: ‘The Love Witch,’ Hell-Bent on Capturi...",None,None,"A Dark Seductress, Hell-Bent on Capturing Hearts",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None
157,Ali and Nino,Ali and Nino,0,en,,Muslim prince Ali and Georgian aristocrat Nino...,10.108,100,0,2016-01-27,...,"Review: ‘Ali and Nino,’ a Love Story Set Again...",None,None,None,None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None


In [131]:
# Drop "byline.person" column
df = df.drop('byline.person', axis=1)
df.columns

Index(['title', 'original_title', 'budget', 'original_language', 'homepage',
       'overview', 'popularity', 'runtime', 'revenue', 'release_date',
       'vote_average', 'vote_count', 'genres', 'spoken_languages',
       'production_countries', 'web_url', 'snippet', 'source', 'keywords',
       'pub_date', 'word_count', 'headline.main', 'headline.kicker',
       'headline.content_kicker', 'headline.print_headline', 'headline.name',
       'headline.seo', 'headline.sub', 'byline.original',
       'byline.organization'],
      dtype='object')

In [133]:
# Delete duplicate rows and reset index
df = df.drop_duplicates()
df

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,After Love,After Love,0,en,,Set in the port town of Dover in the South-Eas...,11.242,89,0,2021-06-04,...,359,‘After Love’ Review: The Other Woman,Critic’s Pick,None,After Love,None,None,None,By Beatrice Loayza,None
1,After Love,After Love,0,en,,Set in the port town of Dover in the South-Eas...,11.242,89,0,2021-06-04,...,274,Review: Staying Together ‘After Love’ and Regr...,None,None,After Love,None,None,None,By Glenn Kenny,None
2,Alcarràs,Alcarràs,3,ca,https://www.alcarras-film.com/,"In a small village in Catalonia, the peach far...",11.197,115,3324521,2022-04-29,...,306,‘Alcarràs’ Review: Labor of Love,None,None,Alcarràs,None,None,None,By Devika Girish,None
3,The Sound of Christmas,The Sound of Christmas,0,en,,A woman facing eviction just before Christmas ...,1.854,0,0,2022-11-24,...,277,‘The Sound of Christmas’ Review: A Gospel Sing...,None,None,The Sound Of Christmas,None,None,None,By Concepción de León,None
4,The Inspection,The Inspection,0,en,https://a24films.com/films/inspection,"Ellis French is a young, gay Black man, reject...",11.802,95,270613,2022-11-18,...,836,"‘The Inspection’ Review: Boot Camp, a Love Story",critic’s pick,None,"A Few Good Men, Some With Secrets",None,None,None,By A.O. Scott,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,Passengers,Passengers,110000000,en,http://www.passengersmovie.com/,A spacecraft traveling to a distant colony pla...,47.257,116,303144152,2016-12-21,...,792,Review: Two ‘Passengers’ Trapped on a Spaceshi...,None,None,Fasten Your Seat Belts for a 120-Year Ride,None,None,None,By Stephen Holden,None
155,Allied,Allied,85000000,en,http://www.alliedmovie.com/,"In 1942, an intelligence officer in North Afri...",22.753,124,119520023,2016-02-12,...,901,"Review: ‘Allied,’ With Brad Pitt and Marion Co...",None,None,"For This Couple, All’s Ambiguous in Love and War",None,None,None,By A.O. Scott,None
156,The Love Witch,The Love Witch,0,en,http://www.lifeofastar.com,A modern-day witch uses spells and magic to ge...,12.123,121,228894,2016-11-11,...,604,"Review: ‘The Love Witch,’ Hell-Bent on Capturi...",None,None,"A Dark Seductress, Hell-Bent on Capturing Hearts",None,None,None,By A.O. Scott,None
157,Ali and Nino,Ali and Nino,0,en,,Muslim prince Ali and Georgian aristocrat Nino...,10.108,100,0,2016-01-27,...,280,"Review: ‘Ali and Nino,’ a Love Story Set Again...",None,None,None,None,None,None,By Ken Jaworowski,None


In [134]:
# Export data to CSV without the index
df.to_csv('data_result.csv', index=False)